In [1]:
:load AnalysisEvents
import AnalysisEvents

In [136]:
import Data.Colour
import Data.Colour.Names
import Graphics.Rendering.Chart
import Graphics.Rendering.Chart.Grid
import Graphics.Rendering.Chart.Backend.Diagrams
import Control.Lens hiding (at, (.>), (|>), elements, assign)
import Data.Default.Class
import Data.Colour
import Data.Colour.Names
import Data.List

import Data.Csv
import qualified Data.ByteString.Char8 as C
import qualified Data.ByteString as B
import qualified Data.ByteString.Lazy as BL
import qualified Data.Vector as V
import Statistics.Quantile

dataFile = "/Users/argyris/work/datastore/experiment2/datac.txt"

In [107]:
mkPoints'' :: [Colour Double] -> String -> String -> String -> [[(Double, Double)]] -> Layout Double Double
mkPoints'' cs xtitle ytitle title valss = layout
  where
    plot c vals = toPlot (plot_points_style .~ filledCircles 3 (c `withOpacity` 0.6)
              $ plot_points_values .~ vals
              $ def )   
    layout = layout_plots .~ [plot c vals | (c, vals) <- zip cs valss]
           $ layout_title  .~ title
           $ layout_x_axis . laxis_style . axis_label_style . font_size .~ 14.0
           $ layout_y_axis . laxis_style . axis_label_style . font_size .~ 14.0
           $ layout_x_axis . laxis_title_style . font_size .~ 13.0
           $ layout_x_axis . laxis_title .~ xtitle
           $ layout_y_axis . laxis_title_style . font_size .~ 13.0
           $ layout_y_axis . laxis_title .~ ytitle
           $ layout_legend .~ Just (legend_label_style . font_size .~ 16.0 $ def)
           $ def
                      
mkLine'' :: [Colour Double] -> String -> String -> String -> [[(Double, Double)]] -> Layout Double Double
mkLine'' cs xtitle ytitle title tvalss = layout
  where
    plot c tvals =
        toPlot
            (  plot_lines_values .~ [tvals]
             $ plot_lines_style . line_color .~ (c `withOpacity` 0.9)
             $ plot_lines_style . line_width .~ 2.0
             $ def)
    layout = layout_plots .~ [plot c vals | (c, vals) <- zip cs tvalss]
           $ layout_title  .~ title
           $ layout_x_axis . laxis_style . axis_label_style . font_size .~ 14.0
           $ layout_y_axis . laxis_style . axis_label_style . font_size .~ 14.0
           $ layout_x_axis . laxis_title_style . font_size .~ 13.0
           $ layout_x_axis . laxis_title .~ xtitle
           $ layout_y_axis . laxis_title_style . font_size .~ 13.0
           $ layout_y_axis . laxis_title .~ ytitle
           $ layout_legend .~ Just (legend_label_style . font_size .~ 16.0 $ def)
           $ def
           
mkHeatMap' :: String -> String -> [(Double, Double, Double, Double)] -> Layout Double Double
mkHeatMap' xtitle ytitle vals = layout 
  where
     plot = toPlot ( area_spots_4d_values .~ vals
                   $ area_spots_4d_max_radius .~ 10
                   $ def)
     layout = layout_plots .~ [plot]
           $ layout_title  .~ ""
           $ layout_x_axis . laxis_style . axis_label_style . font_size .~ 18.0
           
           $ layout_y_axis . laxis_style . axis_label_style . font_size .~ 18.0
           $ layout_x_axis . laxis_title_style . font_size .~ 20.0
           $ layout_x_axis . laxis_title .~ xtitle
           $ layout_y_axis . laxis_title_style . font_size .~ 20.0
           $ layout_legend .~ Just (legend_label_style . font_size .~ 16.0 $ def)
           $ layout_y_axis . laxis_title .~ ytitle
           $ def

In [125]:
{-# LANGUAGE OverloadedStrings #-}

newtype Weight = Weight { unW :: Maybe Double } deriving (Eq, Show)
data Group = Side | Middle deriving (Show, Eq)
data SPos = SPos {ntray :: Int, np :: Int } deriving (Eq, Show)
data Sample = S1 | S2 deriving (Show, Eq)

data SampleP = SampleP { weight :: Weight, 
                       group ::  Group,
                       pos :: SPos,
                       sample :: Sample} deriving (Eq, Show)
                       
                       
instance FromField Weight where
  parseField s 
    | s == "NA" = pure (Weight {unW = Nothing})
    | otherwise = pure (Weight {unW = Just $ read (C.unpack s)})
    
instance FromField Group where
  parseField s
     | s == "M" = pure Middle
     | s == "S" = pure Side
  
instance FromField SPos where
  parseField s = pure (SPos {ntray=read $ C.unpack st, np = read $ (tail $ C.unpack sn)})
    where 
     (st, sn) = B.splitAt 1 s
     
instance FromField Sample where
  parseField s 
    | s == "23" = pure S1
    | s == "27" = pure S2
    
instance FromNamedRecord SampleP where
  parseNamedRecord r = SampleP <$> r .: "weight"
                               <*> r .: "group"
                               <*> r .: "position"
                               <*> r .: "days"
                               
readSamples :: FilePath -> IO [SampleP]
readSamples fin = do
  csvData <- BL.readFile fin
  case decodeByName csvData of
    Left err -> error err
    Right (_, v) -> return $ V.toList v

getTCoords :: Int -> (Int, Int)
getTCoords 1 = (0, 0)
getTCoords 2 = (0, 1)
getTCoords 3 = (1, 0)
getTCoords 4 = (1, 1)
getTCoords _ = undefined

getSPos :: SPos -> (Int, Int)
getSPos SPos {ntray=nt, np=np} = ((fst tray * 4 + fst tcoords+1)*(-1), snd tray * 6 + snd tcoords+1)
  where
    ts = (4, 6)
    tcoords = (np-1) `quotRem` (snd ts)
    tray = getTCoords nt
    
sample1 ss = [s  | s@SampleP {sample=S1} <- ss]

sample2 ss = [s  | s@SampleP {sample=S2} <- ss]

getW :: SampleP -> Double
getW SampleP {weight=w} = case unW w of 
   Just d -> d
   Nothing -> 0.0

--convenience, for plotting
toDs :: (Int, Int) -> (Double, Double)
toDs (x, y) = (fromIntegral x, fromIntegral y)

toN :: Sample -> Double
toN S1 = 1.0
toN S2 = 2.0

In [126]:
[getSPos SPos{ntray=t,np=p} | t <- [1..4], p <- [1..12]]

[(-1,1),(-1,2),(-1,3),(-1,4),(-1,5),(-1,6),(-2,1),(-2,2),(-2,3),(-2,4),(-2,5),(-2,6),(-1,7),(-1,8),(-1,9),(-1,10),(-1,11),(-1,12),(-2,7),(-2,8),(-2,9),(-2,10),(-2,11),(-2,12),(-5,1),(-5,2),(-5,3),(-5,4),(-5,5),(-5,6),(-6,1),(-6,2),(-6,3),(-6,4),(-6,5),(-6,6),(-5,7),(-5,8),(-5,9),(-5,10),(-5,11),(-5,12),(-6,7),(-6,8),(-6,9),(-6,10),(-6,11),(-6,12)]

In [127]:
sps <- readSamples "/Users/argyris/work/datastore/experiment2/datac.txt"

In [128]:
sample1 sps

[SampleP {weight = Weight {unW = Just 0.1388}, group = Middle, pos = SPos {ntray = 1, np = 1}, sample = S1},SampleP {weight = Weight {unW = Just 5.15e-2}, group = Middle, pos = SPos {ntray = 1, np = 2}, sample = S1},SampleP {weight = Weight {unW = Just 0.192}, group = Middle, pos = SPos {ntray = 1, np = 3}, sample = S1},SampleP {weight = Weight {unW = Just 6.82e-2}, group = Middle, pos = SPos {ntray = 1, np = 4}, sample = S1},SampleP {weight = Weight {unW = Just 0.1585}, group = Middle, pos = SPos {ntray = 1, np = 5}, sample = S1},SampleP {weight = Weight {unW = Just 0.1766}, group = Middle, pos = SPos {ntray = 1, np = 6}, sample = S1},SampleP {weight = Weight {unW = Just 0.1407}, group = Middle, pos = SPos {ntray = 2, np = 7}, sample = S1},SampleP {weight = Weight {unW = Just 0.1728}, group = Middle, pos = SPos {ntray = 2, np = 8}, sample = S1},SampleP {weight = Weight {unW = Just 6.93e-2}, group = Middle, pos = SPos {ntray = 2, np = 9}, sample = S1},SampleP {weight = Weight {unW = Ju

In [129]:
heatPoints = [(snd ps, fst ps, getW sp, toN (sample sp)) | sp <- sps, let ps = toDs $ getSPos (pos sp)]

In [130]:
heatPoints

[(1.0,-1.0,0.1388,1.0),(2.0,-1.0,5.15e-2,1.0),(3.0,-1.0,0.192,1.0),(4.0,-1.0,6.82e-2,1.0),(5.0,-1.0,0.1585,1.0),(6.0,-1.0,0.1766,1.0),(7.0,-2.0,0.1407,1.0),(8.0,-2.0,0.1728,1.0),(9.0,-2.0,6.93e-2,1.0),(10.0,-2.0,0.1802,1.0),(11.0,-2.0,9.16e-2,1.0),(12.0,-2.0,0.1125,1.0),(1.0,-5.0,4.8e-2,1.0),(2.0,-5.0,0.1353,1.0),(3.0,-5.0,0.1273,1.0),(4.0,-5.0,0.1047,1.0),(5.0,-5.0,0.1418,1.0),(6.0,-5.0,0.1017,1.0),(7.0,-6.0,0.0,1.0),(8.0,-6.0,0.1001,1.0),(9.0,-6.0,8.88e-2,1.0),(10.0,-6.0,0.1066,1.0),(11.0,-6.0,4.87e-2,1.0),(12.0,-6.0,7.98e-2,1.0),(1.0,-2.0,0.1699,2.0),(2.0,-2.0,0.5355,2.0),(3.0,-2.0,0.4113,2.0),(4.0,-2.0,0.3949,2.0),(5.0,-2.0,0.5437,2.0),(6.0,-2.0,0.29,2.0),(7.0,-1.0,0.4915,2.0),(8.0,-1.0,0.4416,2.0),(9.0,-1.0,0.4039,2.0),(10.0,-1.0,0.4538,2.0),(11.0,-1.0,0.5868,2.0),(12.0,-1.0,0.3896,2.0),(1.0,-6.0,0.2434,2.0),(2.0,-6.0,0.2741,2.0),(3.0,-6.0,0.2398,2.0),(4.0,-6.0,0.2641,2.0),(5.0,-6.0,0.4206,2.0),(6.0,-6.0,0.1743,2.0),(7.0,-5.0,0.0,2.0),(8.0,-5.0,0.1712,2.0),(9.0,-5.0,0.2773,2.0),(1

In [131]:
plotHistsGridR 1 [mkHeatMap' "x" "y" heatPoints]

In [163]:
groupToD :: Group -> Double
groupToD Side = 1.0
groupToD Middle = 2.0

data Distr = Distr { g :: Double, points :: [Double]}
    
getDistrs :: [SampleP] -> (Distr, Distr)
getDistrs sps = (d1, d2) 
  where
    d1 = Distr {g=groupToD Side, points=[getW sp | sp@SampleP {group=Side} <- sps, getW sp > 0.0]}
    d2 = Distr {g=groupToD Middle, points=[getW sp | sp@SampleP {group=Middle} <- sps, getW sp > 0.0]}

sampleToCandles :: [SampleP] -> [Candle Double Double]
sampleToCandles sps = [defCand, toCandle d1, toCandle d2, defCand1]
  where 
    (d1, d2) = getDistrs sps 
    toCandle d = Candle { candle_x = g d,
                candle_low = weightedAvg 0 5 ps,
                candle_open = weightedAvg 1 5 ps,
                candle_mid = weightedAvg 2 5 ps,
                candle_close = weightedAvg 3 5 ps,
                candle_high = weightedAvg 4 5 ps } 
            where ps = V.fromList (points d)
            
    defCand = Candle { candle_x = 0.0, candle_low=0.0, candle_open=0.0, candle_mid=0.0, candle_close=0.0, candle_high=0.0}
    defCand1 = Candle { candle_x = 3.0, candle_low=0.0, candle_open=0.0, candle_mid=0.0, candle_close=0.0, candle_high=0.0}

In [164]:
mkCandlePlot :: String -> String -> [Candle Double Double] -> Layout Double Double
mkCandlePlot xtitle ytitle vals = layout 
  where
     plot = toPlot ( plot_candle_values .~ vals
                   $ def)
     layout = layout_plots .~ [plot]
           $ layout_title  .~ ""
           $ layout_x_axis . laxis_style . axis_label_style . font_size .~ 18.0
           
           $ layout_y_axis . laxis_style . axis_label_style . font_size .~ 18.0
           $ layout_x_axis . laxis_title_style . font_size .~ 20.0
           $ layout_x_axis . laxis_title .~ xtitle
           $ layout_y_axis . laxis_title_style . font_size .~ 20.0
           $ layout_legend .~ Just (legend_label_style . font_size .~ 16.0 $ def)
           $ layout_y_axis . laxis_title .~ ytitle
           $ def
    



In [167]:
-- two candles one for each group
plotHistsGridR 1 [mkCandlePlot "group" "fresh weights (g)" (sampleToCandles (sample2 sps))]

In [168]:
plotHistsGridR 1 [mkCandlePlot  "group" "fresh weights (g)" (sampleToCandles (sample1 sps))]

### Experiment 2 (WT + p7/9)

In [181]:
toCandle :: Double -> V.Vector Double -> Candle Double Double
toCandle x ps = Candle {candle_x = x,
                candle_low = weightedAvg 0 5 ps,
                candle_open = weightedAvg 1 5 ps,
                candle_mid = weightedAvg 2 5 ps,
                candle_close = weightedAvg 3 5 ps,
                candle_high = weightedAvg 4 5 ps }

mkCandlePlot' :: String -> String -> [[Double]] -> Layout Double Double
mkCandlePlot' xtitle ytitle valss = layout 
  where
     plot = toPlot ( plot_candle_values .~ (map (\(x, vals) -> toCandle (fromIntegral x) (V.fromList vals)) (zip [1..length valss] valss))
                   $ def)
     layout = layout_plots .~ [plot]
           $ layout_title  .~ ""
           $ layout_x_axis . laxis_style . axis_label_style . font_size .~ 18.0
           
           $ layout_y_axis . laxis_style . axis_label_style . font_size .~ 18.0
           $ layout_x_axis . laxis_title_style . font_size .~ 20.0
           $ layout_x_axis . laxis_title .~ xtitle
           $ layout_y_axis . laxis_title_style . font_size .~ 20.0
           $ layout_legend .~ Just (legend_label_style . font_size .~ 16.0 $ def)
           $ layout_y_axis . laxis_title .~ ytitle
           $ def

In [221]:
{-# LANGUAGE OverloadedStrings #-}

---shelf,tray,location,mass,group,genotype

data Shelf = T | B deriving (Show)

data Gen = WT | P79 deriving (Show)

data Loc = Loc {shelf :: Shelf, tray :: Int, pos :: Int} deriving (Show)

data Point = Point { loc :: Loc, 
                     mass :: Weight, 
                     group :: Group, 
                     genotype :: Gen} deriving (Show)

instance FromField Shelf where
   parseField s 
     | s == "T" = pure T
     | s == "B" = pure B
     | otherwise = error "cant parse shelf"
     
parseLoc s t p = Loc <$> s <*> t <*> p

instance FromField Gen where
  parseField s 
    | s == "WT" = pure WT
    | s == "P79" = pure P79
    | otherwise = error "cant parse gen"
                        
instance FromNamedRecord Point where
    parseNamedRecord r = Point <$> parseLoc (r .: "shelf") (r .: "tray") (r .: "location")
                               <*> r .: "mass"
                               <*> r .: "group"
                               <*> r .: "genotype"
                               
readSamples' :: FilePath -> IO [Point]
readSamples' fin = do
  csvData <- BL.readFile fin
  case decodeByName csvData of
    Left err -> error err
    Right (_, v) -> return $ V.toList v
    
isDead :: Point -> Bool
isDead Point {mass=Weight{unW=Nothing}} = True
isDead _ = False

In [224]:
sps <- readSamples' "/Users/argyris/work/datastore/experiment3/data/sample1/041217.txt"

In [225]:
length (filter isDead sps)

16

In [226]:
length sps

72

In [228]:
sps1 <- readSamples' "/Users/argyris/work/datastore/experiment3/data/sample2/061217.txt"

In [229]:
length (filter isDead sps1)

18

In [230]:
length sps1

72